<a href="https://colab.research.google.com/github/KinshukTr/RAGwithKG/blob/main/RAGwithKG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain_google_genai langchain-groq groq langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs sentence-transformers



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━

In [2]:
from google.colab import userdata
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
HF_TOKEN=userdata.get('HF_TOKEN')




In [3]:
NEO4J_URI="neo4j+s://e1591df3.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="IdSPF35BFLA-b-13hVaTv-77CCX_hap2bgqPVmjvLx0"

In [4]:
import os

In [5]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [6]:
from langchain_community.graphs import Neo4jGraph

In [7]:
graph=Neo4jGraph()

<ipython-input-7-95571a775733>:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph()


In [8]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Optimus Prime").load()


In [9]:
len(raw_documents)

25

In [10]:
raw_documents[ :3]

[Document(metadata={'title': 'Optimus Prime', 'summary': "Optimus Prime, also known in Japan as Convoy, is a fictional character and the main protagonist of the Transformers franchise. Generally depicted as a brave and noble leader, Optimus Prime is the supreme commander of the Autobots in their fight against the Decepticons. The Transformers characters were developed for the American market after Hasbro representatives visited the 1983 Tokyo Toy Show. The characters were modified and the coloring was changed; notably, Optimus Prime was colored red, chrome, and dark blue. The popularity of the Transformers toys resulted in comics, movies, and a TV series.\nIn the Transformers mythology, Optimus Prime is a Cybertronian, a member of an extraterrestrial species of sentient self-configuring modular robotic lifeforms (e.g., cars and other objects), a blend of biological evolution and technological engineering. He is the primary hero of the story, opposing the Decepticon leader Megatron.\nDu

In [11]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])


In [12]:
from langchain_groq import ChatGroq  # Import Groq's chat model
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [13]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [14]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)


graph_documents

[GraphDocument(nodes=[Node(id='Optimus Prime', type='Character', properties={}), Node(id='Convoy', type='Alias', properties={}), Node(id='Transformers Franchise', type='Franchise', properties={}), Node(id='Autobots', type='Group', properties={}), Node(id='Decepticons', type='Group', properties={}), Node(id='Hasbro', type='Company', properties={}), Node(id='Tokyo Toy Show', type='Event', properties={}), Node(id='Cybertronian', type='Species', properties={}), Node(id='Megatron', type='Character', properties={}), Node(id='Peter Cullen', type='Actor', properties={}), Node(id='Garry Chalk', type='Actor', properties={}), Node(id='Neil Kaplan', type='Actor', properties={}), Node(id='David Kaye', type='Actor', properties={}), Node(id='Alan Tudyk', type='Actor', properties={}), Node(id='Chris Hemsworth', type='Actor', properties={}), Node(id='Orion Pax', type='Alias', properties={}), Node(id='Matrix Of Leadership', type='Artifact', properties={}), Node(id='Unicron', type='Character', properties

In [15]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [16]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [17]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [18]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass


In [19]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [20]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [21]:
from typing import Tuple, List, Optional


from langchain_community.vectorstores import Neo4jVector

In [22]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_index = Neo4jVector.from_existing_graph(
    hf_embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

<ipython-input-22-f64ff5b77d8a>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
from pydantic import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate


In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [26]:
entity_chain = prompt | llm.with_structured_output(Entities)


entity_chain.invoke({"question": "Where was Orion Pax born?"}).names

['Orion Pax']

In [27]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [28]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [29]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result


In [30]:
print(structured_retriever("Who is Optimus Prime?"))

<ipython-input-28-bcc56ddd5940>:3: LangChainDeprecationWarning: The function `remove_lucene_chars` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the function exists in the :meth:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :meth:`~langchain-neo4j` and import as `from :meth:`~langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars``.
  words = [el for el in remove_lucene_chars(input).split() if el]


Primus - BESTOWS -> Matrix of Leadership
Primus - CHARACTER -> Transformers One
Optimus Prime - CREATED_BY -> Primus
Thirteen Primes - CREATED_BY -> Primus
Matrix of Leadership - GIVEN_BY -> Primus
aed1b3ac5dcb546f9a9c930f16d8a1cdcd263ca5 - HAS_ENTITY -> Primus
268146911474a13bdec7d54008e37b2688071876 - HAS_ENTITY -> Primus
88d5dd59f48d03c7f287d15a2487654c45843f5d - HAS_ENTITY -> Primus
Optimus Prime - PART_OF -> Transformers franchise
Optimus Prime - PART_OF -> Transformers
Optimus Prime - PART_OF -> Transformers Franchise
Optimus Prime - ALSO_KNOWN_AS -> Convoy
Optimus Prime - ALSO_KNOWN_AS -> Orion Pax
Optimus Prime - MAIN_PROTAGONIST_OF -> Transformers franchise
Optimus Prime - SUPREME_COMMANDER_OF -> Autobots
Optimus Prime - COLORED_AS -> red, chrome, and dark blue
Optimus Prime - IS_A -> Cybertronian
Optimus Prime - OPPOSES -> Megatron
Optimus Prime - PORTRAYED_BY -> Neil Kaplan
Optimus Prime - PORTRAYED_BY -> Chris Hemsworth
Optimus Prime - PORTRAYED_BY -> Garry Chalk
Optimus Pr

In [31]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [32]:

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [33]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [34]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [35]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [36]:
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough, RunnableLambda, RunnableParallel
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [37]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [38]:
prompt = ChatPromptTemplate.from_template(template)

In [39]:
    chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)


chain.invoke({"question": "Which planet did Orion Pax belong to?"})


Search query: Which planet did Orion Pax belong to?


'Cybertron.'

In [40]:
chain.invoke(
    {
        "question": "Where was he working?"

    }
)

Search query: Where was he working?


'He was working in Iacon, as a young data clerk, under the wing of Alpha Trion.'